In [8]:
import pandas as pd
import numpy as np
from pathlib import Path

print("Process cwd:", Path.cwd())

# Load combined dataset directly
data_dir = (Path.cwd() / ".." / "data" / "Cleaned Data").resolve()
combined_path = data_dir / "combined_prem_xg_2020_2026.csv"

print(f"Loading combined dataset from: {combined_path}")
print(f"Exists: {combined_path.exists()}")

if not combined_path.exists():
    print("ERROR: combined_prem_xg_2020_2026.csv not found!")
else:
    # Load the combined dataset
    pricing_df = pd.read_csv(combined_path, parse_dates=["date"])
    print(f"Loaded {len(pricing_df)} rows")
    print(f"Columns: {pricing_df.columns.tolist()}")

    # ===== CONSTRUCT FEATURES FOR REGRESSION =====
    
    # Compute derived metrics for each match
    pricing_df["pts"] = pricing_df["result"].map({"W": 3, "D": 1, "L": 0})
    pricing_df["xG_diff"] = pricing_df["xG"] - pricing_df["xGA"]
    pricing_df["clean_sheet"] = (pricing_df["goals_against"] == 0).astype(int)
    pricing_df["card_points"] = pricing_df["yellow"] + 2 * pricing_df["red"]

    # Compute OppStrength (opponent's average goals_for as proxy)
    opp_strength = pricing_df.groupby("opponent")["goals_for"].transform("mean")
    pricing_df["opp_strength"] = opp_strength.fillna(0)

    # ===== RUN OLS REGRESSION TO GET DATA-DRIVEN COEFFICIENTS =====
    
    print(f"\n===== CHECKING DATA FOR REGRESSION =====")
    print(f"Total rows: {len(pricing_df)}")
    print(f"Rows with xG: {pricing_df['xG'].notna().sum()}")
    print(f"Rows with xG_diff: {pricing_df['xG_diff'].notna().sum()}")
    print(f"Rows with pts: {pricing_df['pts'].notna().sum()}")
    
    # Select features for regression (only rows with complete data)
    feature_cols = ["xG_diff", "clean_sheet", "card_points", "opp_strength"]
    regression_df = pricing_df[feature_cols + ["pts"]].dropna()
    
    print(f"\n===== RUNNING OLS REGRESSION =====")
    print(f"Features used: {feature_cols}")
    print(f"Regression dataset: {len(regression_df)} rows (after dropping NaN)")
    
    if len(regression_df) < 50:
        print("⚠️ ERROR: Not enough data for regression!")
    else:
        X = regression_df[feature_cols].values
        y = regression_df["pts"].values
        
        # Add intercept column
        X_with_intercept = np.c_[np.ones(len(X)), X]
        
        # Solve OLS: minimize ||y - Xθ||²
        theta, residuals, rank, s = np.linalg.lstsq(X_with_intercept, y, rcond=None)
        
        # Unpack coefficients
        zeta_hat = theta[0]           # intercept
        beta_hat = theta[1]           # xG_diff coefficient
        gamma_hat = theta[2]          # clean_sheet coefficient
        delta_hat_raw = theta[3]      # card_points
        epsilon_hat_raw = theta[4]    # opp_strength
        
        print("\n===== REGRESSION COEFFICIENTS (RAW) =====")
        print(f"Intercept (ζ): {zeta_hat:.4f}")
        print(f"β (xG_diff): {beta_hat:.4f}")
        print(f"γ (clean_sheet): {gamma_hat:.4f}")
        print(f"δ_raw (card_points): {delta_hat_raw:.4f}")
        print(f"ε_raw (opp_strength): {epsilon_hat_raw:.4f}")
        
        # Take absolute values for delta and epsilon (we use minus sign in formula)
        delta_hat = abs(delta_hat_raw)
        epsilon_hat = abs(epsilon_hat_raw)
        
        # ===== RESCALE FOR PRICE STABILITY =====
        shrink = 0.2  # Tune this for price stability
        
        alpha = 0.5 * shrink  # Add WDL impact
        beta = beta_hat * shrink
        gamma = gamma_hat * shrink
        delta = delta_hat * shrink
        epsilon = epsilon_hat * shrink
        zeta = zeta_hat * shrink
        
        print("\n===== SCALED COEFFICIENTS FOR PRICING FORMULA =====")
        print(f"α (WDL): {alpha:.4f}")
        print(f"β (xG_diff): {beta:.4f}")
        print(f"γ (clean_sheet): {gamma:.4f}")
        print(f"δ (card_points): {delta:.4f}")
        print(f"ε (opp_strength): {epsilon:.4f}")
        print(f"ζ (intercept): {zeta:.4f}")

        # ===== APPLY PRICING FORMULA WITH DATA-DRIVEN COEFFICIENTS =====

        # Initialize starting price
        P0 = 100

        # Compute match-level price change
        pricing_df["delta_P"] = (
            alpha * pricing_df["pts"] 
            + beta * pricing_df["xG_diff"].fillna(0)
            + gamma * pricing_df["clean_sheet"]
            - delta * pricing_df["card_points"]
            + epsilon * pricing_df["opp_strength"]
            + zeta
        )

        # Compute cumulative price by team
        pricing_df = pricing_df.sort_values(["team", "date"]).reset_index(drop=True)
        pricing_df["price"] = pricing_df.groupby("team")["delta_P"].cumsum() + P0

        print("\n===== PRICING WITH DATA-DRIVEN COEFFICIENTS =====")
        display(pricing_df[["date", "team", "opponent", "pts", "xG_diff", "clean_sheet", "card_points", "delta_P", "price"]].head(20))

        # Export full pricing data to CSV
        pricing_df.to_csv("pricing_output_data_driven.csv", index=False)
        print("Saved full pricing data to pricing_output_data_driven.csv")

        # ===== GET FINAL PRICES FOR TARGET TEAMS =====
        
        # Get final prices for all teams
        final_prices = pricing_df.groupby("team").agg({
            "price": "last",
            "pts": "sum",
            "xG_diff": "mean",
            "clean_sheet": "sum",
            "card_points": "sum",
            "delta_P": "mean"
        }).reset_index()

        final_prices.columns = ["team", "final_price", "total_pts", "avg_xG_diff", "total_clean_sheets", "total_card_points", "avg_delta_P"]
        final_prices = final_prices.sort_values("final_price", ascending=False)

        print("\n===== FINAL TEAM STOCK PRICES (DATA-DRIVEN) =====\n")
        display(final_prices)

        # Filter for target teams
        target_teams = [
            "Arsenal", "Aston Villa", "Bournemouth", "Brentford", "Brighton and Hove Albion", 
            "Burnley", "Chelsea", "Crystal Palace", "Everton", "Fulham", 
            "Leeds United", "Liverpool", "Manchester City", "Manchester United", 
            "Newcastle United", "Nottingham Forest", "Sunderland", "Tottenham Hotspur", 
            "West Ham United", "Wolverhampton Wanderers"
        ]

        def normalize_team_name(name):
            return name.lower().strip()

        final_prices["team_normalized"] = final_prices["team"].apply(normalize_team_name)
        target_teams_normalized = [normalize_team_name(t) for t in target_teams]

        target_final_prices = final_prices[final_prices["team_normalized"].isin(target_teams_normalized)].copy()
        target_final_prices = target_final_prices.sort_values("final_price", ascending=False)

        print("\n===== TARGET TEAMS STOCK PRICES =====\n")
        display(target_final_prices[["team", "final_price", "total_pts", "avg_xG_diff", "total_clean_sheets", "total_card_points"]])

        target_final_prices.to_csv("target_teams_pricing_data_driven.csv", index=False)
        print("\nSaved target teams to target_teams_pricing_data_driven.csv")

        print("\n===== SUMMARY =====")
        print(f"Total teams: {len(final_prices)}")
        print(f"Target teams found: {len(target_final_prices)}")
        print(f"\nPrice range (all): {final_prices['final_price'].min():.2f} - {final_prices['final_price'].max():.2f}")
        print(f"Price range (target): {target_final_prices['final_price'].min():.2f} - {target_final_prices['final_price'].max():.2f}")

Process cwd: /Users/saarj/Documents/The Drive/UNIL Masters/Year 1/Autumn Semester/Datascience and Advanced Programming/fantasy-football-stock-market-simulator/src
Loading combined dataset from: /Users/saarj/Documents/The Drive/UNIL Masters/Year 1/Autumn Semester/Datascience and Advanced Programming/fantasy-football-stock-market-simulator/data/Cleaned Data/combined_prem_xg_2020_2026.csv
Exists: True
Loaded 4020 rows
Columns: ['date', 'team', 'opponent', 'home_away', 'goals_for', 'goals_against', 'yellow', 'red', 'result', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep', 'deep_allowed', 'xpts']

===== CHECKING DATA FOR REGRESSION =====
Total rows: 4020
Rows with xG: 2090
Rows with xG_diff: 2090
Rows with pts: 4020

===== RUNNING OLS REGRESSION =====
Features used: ['xG_diff', 'clean_sheet', 'card_points', 'opp_strength']
Regression dataset: 2090 rows (after dropping NaN)

===== REGRESSION COEFFICIENTS (RAW) =====
Intercept (ζ): 0.6159
β (xG_diff): 0.3964
γ (clean_sheet): 1.1

,date,team,opponent,pts,xG_diff,clean_sheet,card_points,delta_P,price
0,2020-09-12,Arsenal,Fulham,3,2.036543,1,2,0.899363,100.899363
1,2020-09-19,Arsenal,West Ham United United,3,-0.734750,0,0,0.462631,101.361994
2,2020-09-28,Arsenal,Liverpool,0,-1.560170,0,2,0.065570,101.427564
3,2020-10-04,Arsenal,Sheffield United,3,0.522137,0,0,0.602890,102.030454
4,2020-10-17,Arsenal,Manchester City,0,-0.591908,0,1,0.131215,102.161669
5,2020-10-25,Arsenal,Leicester City,0,0.003268,0,3,0.228471,102.390140
6,2020-11-01,Arsenal,Manchester United,3,0.603959,1,3,0.779602,103.169742
7,2020-11-08,Arsenal,Aston Villa,0,-0.595160,0,0,0.159919,103.329661
8,2020-11-22,Arsenal,Leeds United,1,-1.250922,1,2,0.463521,103.793183
9,2020-11-29,Arsenal,Wolverhampton Wanderers,0,-0.888400,0,3,0.149055,103.942238


Saved full pricing data to pricing_output_data_driven.csv

===== FINAL TEAM STOCK PRICES (DATA-DRIVEN) =====



,team,final_price,total_pts,avg_xG_diff,total_clean_sheets,total_card_points,avg_delta_P
12,Manchester City,225.217219,452,1.421130,84,273,0.622971
11,Liverpool,214.235070,412,1.056628,74,319,0.568334
0,Arsenal,209.908652,403,0.723408,77,329,0.546809
6,Chelsea,199.490249,337,0.592288,68,441,0.494976
13,Manchester United,188.642035,327,NaN,58,410,0.441005
1,Aston Villa,187.238252,313,-0.022155,59,431,0.434021
17,Tottenham Hotspur,186.852094,315,NaN,55,401,0.432100
14,Newcastle United,186.136256,303,NaN,57,369,0.428539
4,Brighton and Hove Albion,182.357199,279,NaN,51,392,0.409737
18,West Ham United,178.724683,266,NaN,41,336,0.391665



===== TARGET TEAMS STOCK PRICES =====



,team,final_price,total_pts,avg_xG_diff,total_clean_sheets,total_card_points
12,Manchester City,225.217219,452,1.421130,84,273
11,Liverpool,214.235070,412,1.056628,74,319
0,Arsenal,209.908652,403,0.723408,77,329
6,Chelsea,199.490249,337,0.592288,68,441
13,Manchester United,188.642035,327,NaN,58,410
1,Aston Villa,187.238252,313,-0.022155,59,431
17,Tottenham Hotspur,186.852094,315,NaN,55,401
14,Newcastle United,186.136256,303,NaN,57,369
4,Brighton and Hove Albion,182.357199,279,NaN,51,392
18,West Ham United,178.724683,266,NaN,41,336



Saved target teams to target_teams_pricing_data_driven.csv

===== SUMMARY =====
Total teams: 28
Target teams found: 20

Price range (all): 105.12 - 225.22
Price range (target): 105.12 - 225.22


In [10]:
# Add this at the end of your existing code, before filtering for target teams

# ===== DEBUG: FIND MISSING TEAMS =====
print("\n===== ALL TEAMS IN DATASET =====")
all_teams = final_prices["team"].unique()
print(sorted(all_teams))

print("\n===== SEARCHING FOR TOTTENHAM AND BRIGHTON =====")
for team in all_teams:
    if "tottenham" in team.lower() or "brighton" in team.lower() or "spurs" in team.lower():
        print(f"Found: '{team}'")

# ===== UPDATE TARGET TEAMS WITH EXACT NAMES =====
target_teams = [
    "Arsenal", "Aston Villa", "Bournemouth", "Brentford", "Brighton and Hove Albion",  # Try without "and Hove Albion"
    "Burnley", "Chelsea", "Crystal Palace", "Everton", "Fulham", 
    "Leeds United", "Liverpool", "Manchester City", "Manchester United", 
    "Newcastle United", "Nottingham Forest", "Sunderland", "Tottenham Hotspur",
    "West Ham United", "Wolverhampton Wanderers"
]

# Also try these alternative spellings
alternative_names = [
    "Tottenham", "Brighton Hove and Albion", "Brighton & Hove Albion"
]

def normalize_team_name(name):
    return name.lower().strip()

final_prices["team_normalized"] = final_prices["team"].apply(normalize_team_name)
target_teams_normalized = [normalize_team_name(t) for t in target_teams + alternative_names]

target_final_prices = final_prices[final_prices["team_normalized"].isin(target_teams_normalized)].copy()
target_final_prices = target_final_prices.sort_values("final_price", ascending=False)

# Show which teams are still missing
found_teams = set(target_final_prices["team_normalized"].tolist())
missing = [t for t in target_teams if normalize_team_name(t) not in found_teams]
print(f"\n⚠️ Still missing: {missing}")

print("\n===== TARGET TEAMS STOCK PRICES =====\n")
display(target_final_prices[["team", "final_price", "total_pts", "avg_xG_diff", "total_clean_sheets", "total_card_points"]])

target_final_prices.to_csv("target_teams_pricing_data_driven.csv", index=False)
print(f"\nSaved {len(target_final_prices)} teams to target_teams_pricing_data_driven.csv")


===== ALL TEAMS IN DATASET =====
['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton and Hove Albion', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leeds United', 'Liverpool', 'Manchester City', 'Manchester United', 'Newcastle United', 'Nottingham Forest', 'Sunderland', 'Tottenham Hotspur', 'West Ham United', 'Wolverhampton Wanderers', 'ipswich town', 'leicester city', 'luton town', 'norwich city', 'sheffield united', 'southampton', 'watford', 'west brom']

===== SEARCHING FOR TOTTENHAM AND BRIGHTON =====
Found: 'Tottenham Hotspur'
Found: 'Brighton and Hove Albion'

⚠️ Still missing: []

===== TARGET TEAMS STOCK PRICES =====



,team,final_price,total_pts,avg_xG_diff,total_clean_sheets,total_card_points
12,Manchester City,225.217219,452,1.421130,84,273
11,Liverpool,214.235070,412,1.056628,74,319
0,Arsenal,209.908652,403,0.723408,77,329
6,Chelsea,199.490249,337,0.592288,68,441
13,Manchester United,188.642035,327,NaN,58,410
1,Aston Villa,187.238252,313,-0.022155,59,431
17,Tottenham Hotspur,186.852094,315,NaN,55,401
14,Newcastle United,186.136256,303,NaN,57,369
4,Brighton and Hove Albion,182.357199,279,NaN,51,392
18,West Ham United,178.724683,266,NaN,41,336



Saved 20 teams to target_teams_pricing_data_driven.csv
